# Think Bayes

+ [코드모음](https://github.com/AllenDowney/ThinkBayes2/tree/master/code)
+ [thinkbayes2 모듈](https://github.com/AllenDowney/ThinkBayes2/blob/master/code/thinkbayes2.py)

## Ch.4 : 추정2

### 4.1 유로 문제

> 가디언지 2002년 1월 4일 금요일  
> 벨기에 1유로 동전 실험  
> 축을 중심으로 250번 회전 : 앞면 140회, 뒷면 110회  
> + LSE교수 베리 브라이트 : '내가 보기엔 의심스럽다. 만약 동전이 한 쪽으로 기울어지넥 아니라면, 결과가 이렇게 치우칠 확률은 7% 미만이다.'  
> + 데이빗 맥케이 : 이 데이터로 동전이 한 쪽으로 기울었다는 것을 증명할 수 있는가?


<답을 위한 2단계>  
1. 앞면 나올 확률 추정  
2. 데이터가 가설(동전이 기울었다)을 지지하는지 평가  

+ $ x $ : 주어진 동전을 돌렸을 때 앞면이 나오며 넘어갈 확률  
+ $ x $의 값은 동전의 무게 분포 등 물리적 성격에 따라 달라진다.    
+ 동전의 무게가 완벽히 균등하다면 $ x $ : 50%  
+ 한 쪽으로 기울어진 동전이라면 상대적으로 $ x $의 값은 달라진다.  
+ 여기에 베이즈 이론을 적용하여 데이터 관측을 통해 $ x $ 추정 가능  
+ $ Hx $ : 앞면이 나올 확률은 $ x$%  
+ $ Hx $의 확률은 모든 $x$에 대해서 동일 (균등 사전 분포에서 시작)  

In [3]:
import thinkbayes2
import thinkplot

In [15]:
#우도함수
class Euro(thinkbayes2.Suite):
    """Represents hypotheses about the probability of heads."""

    def Likelihood(self, data, hypo):
        """Computes the likelihood of the data under the hypothesis.
        hypo: integer value of x, the probability of heads (0-100)
        data: string 'H' or 'T'
        """
        x = hypo / 100.0
        if data == 'H':
            return x
        else:
            return 1-x

In [16]:
# 균등 사전분포
def UniformPrior():
    """Makes a Suite with a uniform prior."""
    suite = Euro(range(0, 101))
    return suite

In [17]:
# 삼각 사전분포
def TrianglePrior():
    """Makes a Suite with a triangular prior."""
    suite = Euro()
    for x in range(0, 51):
        suite.Set(x, x)
    for x in range(51, 101):
        suite.Set(x, 100-x) 
    suite.Normalize()
    return suite

In [18]:
#앞면, 뒷면 횟수 정보 업데이트
def RunUpdate(suite, heads=140, tails=110):
    """Updates the Suite with the given number of heads and tails.
    suite: Suite object
    heads: int
    tails: int
    """
    dataset = 'H' * heads + 'T' * tails

    for data in dataset:
        suite.Update(data)

In [19]:
def Summarize(suite):
    """Prints summary statistics for the suite."""
    print(suite.Prob(50))

    print('MLE', suite.MaximumLikelihood())

    print('Mean', suite.Mean())
    print('Median', suite.Percentile(50)) 

    print('5th %ile', suite.Percentile(5)) 
    print('95th %ile', suite.Percentile(95)) 

    print('CI', suite.CredibleInterval(90))

In [20]:
def PlotSuites(suites, root):
    """Plots two suites.
    suite1, suite2: Suite objects
    root: string filename to write
    """
    thinkplot.Clf()
    thinkplot.PrePlot(len(suites))
    thinkplot.Pmfs(suites)

    thinkplot.Save(root=root,
                   xlabel='x',
                   ylabel='Probability',
                   formats=['pdf', 'eps'])


In [21]:
def main():
    # make the priors
    suite1 = UniformPrior()
    suite1.name = 'uniform'

    suite2 = TrianglePrior()
    suite2.name = 'triangle'

    # plot the priors
    PlotSuites([suite1, suite2], 'euro2')

    # update
    RunUpdate(suite1)
    Summarize(suite1)

    RunUpdate(suite2)
    Summarize(suite2)

    # plot the posteriors
    PlotSuites([suite1], 'euro1')
    PlotSuites([suite1, suite2], 'euro3')


In [22]:
if __name__ == '__main__':
    main()

c:\program files\anaconda3\lib\site-packages\matplotlib\axes\_axes.py:545: UserWarning: No labelled objects found. Use label='...' kwarg on individual plots.
  warnings.warn("No labelled objects found. "


Writing euro2.pdf
Writing euro2.eps
0.02097652612954468
MLE 56
Mean 55.952380952380956
Median 56
5th %ile 51
95th %ile 61
CI (51, 61)
0.02384753721469363
MLE 56
Mean 55.74349943859506
Median 56
5th %ile 51
95th %ile 61
CI (51, 61)
Writing euro1.pdf
Writing euro1.eps
Writing euro3.pdf
Writing euro3.eps


균등사전확률에 대한 사후 확률 분포
![uniform](http://postfiles16.naver.net/MjAxNzA5MzBfMTM2/MDAxNTA2NzE4MTM0OTM3.cEmJOxmOMiPbg1dlNfbEb-aDQNWLVDSu3vF9lFomv_og.HOM72MQyd_5ElQqxhaYmNK8rkjvVLJIC1MkRqnzNc24g.PNG.taesiri/euro1.png?type=w2)

### 4.2 사후 확률 요약하기

In [26]:
#요약
suite1 = UniformPrior()
suite1.name = 'uniform'

RunUpdate(suite1)
Summarize(suite1)

0.02097652612954468
MLE 56
Mean 55.952380952380956
Median 56
5th %ile 51
95th %ile 61
CI (51, 61)


결과값 :56  
앞면의 비율인 140/250 * 100 = 0.56(56%)과 일치  
평균 : 55.95  
중위수 : 56  
95% 신뢰구간 : (51, 61)  

사후 신뢰구간이 50%를 포함하고 있지 않음 --> 동전이 평평하지 않다

### 4.3 사전분포 범람

+ 균등 사전 분포는 좋은 선택이 아니다.  
+ 만약 동전이 한 쪽으로 치우쳐 있다면 $x$는 50%를 살짝 벗어날 것이지만  
+ 벨기에 유로 동전은 심각하게 균등하지 않아서 $x$가 10%이거나 90%일 것이라고 생각하기 힘들다.  
+ 50%에 가까운 $x$의 값에 더 높은 확률을 부여하고 극단 값에 낮은 확률을 부여하는 사전확률이 더 자연스럽다.  

In [ ]:
# 삼각 사전분포 (실행x)
def TrianglePrior():
    """Makes a Suite with a triangular prior."""
    suite = Euro()
    for x in range(0, 51):
        suite.Set(x, x)
    for x in range(51, 101):
        suite.Set(x, 100-x) 
    suite.Normalize()
    return suite

균등 사전확률과 삼각 사전 확률  
+ 짙은색 : 균등  
+ 옅은색 : 삼각
![삼각사전분포](http://postfiles10.naver.net/MjAxNzA5MzBfMTEx/MDAxNTA2NzIwMjUzMDc0.M9LgIgO-Xd9nn8rhw78KJS8WTCwWdrlNoAanhWAh0cYg.dt-h2jUdZG-SABikC5kL7EB-6d85hSUJMWpEX8ZyFJ4g.PNG.taesiri/euro2.png?type=w2)

+ 동일한 데이터 셋의 사전확률을 갱신하여 사후 확률을 나타내기

+ 짙은색 : 균등  
+ 옅은색 : 삼각
![사후확률 비교](http://postfiles10.naver.net/MjAxNzA5MzBfMjI3/MDAxNTA2NzIwNDEzMjc3.lxN7rWlPvEV5AK4dJ626OqRaEpiKOlfEkc31koH5qa0g.xCF32AXqbb9tbX25CbTQ8RiEDMY6eDltGsrHdxOhmssg.PNG.taesiri/euro3.png?type=w2)

+ 사전확률이 다름에도 불구하고(균등과 삼각)  사후 확률 분포는 유사하다.  

In [27]:
#요약
suite2 = TrianglePrior()
suite2.name = 'triangle'

RunUpdate(suite2)
Summarize(suite2)

0.02384753721469363
MLE 56
Mean 55.74349943859506
Median 56
5th %ile 51
95th %ile 61
CI (51, 61)


+ MLE, 중간값, 신뢰구간은 균등분포와 동일  
+ 평균은 55.74(삼각사전분포)로 55.95(균등사전분포)와 0.5% 미만의 차이   

**사전 분포 범람**  
: 데이터가 충분하다면 서로 다른 사전 확률을 가지고 시작해도 동일한 사후 확률로 수렴하는 경향

### 4.5 베타 분포

+ 베타 분포는 켤레 사전 분포(conjugate prior, 사후 분포가 사전 분포와 동일)  
+ x에 대한 사전 분포가 베타 분포라면, 사후 확률 역시 베타 분포  
+ 베타 분포의 모양은 $\alpha$와 $\beta$ 두 값에 따라 달라진다.  
+ 사전분포가 alpha와 beta에 대한 분포이고 앞면 h와 뒷면 t의 데이터를 가지고 있다면  
+ 사후확률은 alpha+h와 beta+t에 대한 베타 분포가 될 것. 즉, 두 개를 추가해서 갱신   
+ 이는 베타 분포가 사전 분포로 적합한 경우에만 동작한다.  
+ 베타 분포는 많은 경우에 좋은 추정값으로 사용. 균등 사전 분포의 경우 완벽하게 맞아 떨어진다.  
+ alpha =1, beta=1인 베타 분포는 0부터 1까지 균등

In [31]:
'''
class Beta:
    """Represents a Beta distribution.
    See http://en.wikipedia.org/wiki/Beta_distribution
    """
    def __init__(self, alpha=1, beta=1, label=None):
        """Initializes a Beta distribution."""
        self.alpha = alpha
        self.beta = beta
        self.label = label if label is not None else '_nolegend_'

    def Update(self, data):
        """Updates a Beta distribution.
        data: pair of int (heads, tails)
        """
        heads, tails = data
        self.alpha += heads
        self.beta += tails

    def Mean(self):
        """Computes the mean of this distribution."""
        return self.alpha / (self.alpha + self.beta)
'''

'\nclass Beta:\n    """Represents a Beta distribution.\n    See http://en.wikipedia.org/wiki/Beta_distribution\n    """\n    def __init__(self, alpha=1, beta=1, label=None):\n        """Initializes a Beta distribution."""\n        self.alpha = alpha\n        self.beta = beta\n        self.label = label if label is not None else \'_nolegend_\'\n\n    def Update(self, data):\n        """Updates a Beta distribution.\n        data: pair of int (heads, tails)\n        """\n        heads, tails = data\n        self.alpha += heads\n        self.beta += tails\n\n    def Mean(self):\n        """Computes the mean of this distribution."""\n        return self.alpha / (self.alpha + self.beta)\n'

In [30]:
beta = thinkbayes2.Beta()
beta.Update((140,110))
print(beta.Mean())

0.5595238095238095


+ 사전확률 56%로 이전 결과와 동일

### 결론

+ 동일한 문제를 두 개의 서로 다른 사전 분포로 해결  
+ 데이터 셋이 큰 경우에 사전 분포 범람 현상이 발생   
+ 서로 다른 사전 확률로 시작하여도 데이터를 접하면서 (update) 사후 확률이 수렴  